# Optimizing a Supersonic Aircraft Wing

In [1]:
import random
import numpy as np
from deap import base, creator, tools

# Function to ensure all attributes stay within the defined bounds
def ensure_bounds(ind, low, up):
    for i in range(len(ind)):
        ind[i] = min(max(ind[i], low[i]), up[i])
    return ind

# More realistic placeholders for lift and drag calculations
def aerodynamics(span, aspect_ratio, sweep):
    lift = span * aspect_ratio * np.cos(np.radians(sweep)) / (span * 0.1)
    drag = 100 / lift + span * np.sin(np.radians(sweep)) * 0.1
    return lift, drag

# Evaluation function to minimize drag and maximize lift
def evaluate(ind):
    span, aspect_ratio, sweep = ind
    lift, drag = aerodynamics(span, aspect_ratio, sweep)
    return (drag, -lift)  # Minimize drag, maximize lift (as negative for minimization)

# Mutation function applying Gaussian mutation and ensuring bounds
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < 0.2:  # Increased mutation chance
            individual[i] += random.gauss(0, 3)  # Increased mutation effect
    return ensure_bounds(individual, [25, 1, 15], [65, 15, 65]),

# Blending crossover function and ensuring bounds
def mate(ind1, ind2):
    tools.cxBlend(ind1, ind2, alpha=0.5)
    ensure_bounds(ind1, [30, 2, 20], [60, 12, 60])
    ensure_bounds(ind2, [30, 2, 20], [60, 12, 60])
    return ind1, ind2

# Set up DEAP framework for genetic algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_span", random.uniform, 20, 70)  # Further expanded span range
toolbox.register("attr_aspect_ratio", random.uniform, 10, 20)  # Higher aspect ratio range
toolbox.register("attr_sweep", random.uniform, 10, 25)  # Slightly higher sweep range


toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_span, toolbox.attr_aspect_ratio, toolbox.attr_sweep), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# Main function to execute the genetic algorithm
def main():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    num_generations = 50
    for gen in range(num_generations):
        offspring = list(map(toolbox.clone, toolbox.select(pop, len(pop))))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)
        pop[:] = offspring
        hof.update(pop)
        record = stats.compile(pop)
        print(f"Generation {gen}: {record}")

    print("Best individual is:", hof[0])
    print("With fitness:", hof[0].fitness.values)
    span, aspect_ratio, sweep = hof[0]
    lift, drag = aerodynamics(span, aspect_ratio, sweep)
    print(f"Optimized Wing Configuration: Span={span:.2f}m, Aspect Ratio={aspect_ratio:.2f}, Sweep={sweep:.2f} degrees")
    print(f"Lift={lift:.2f}, Drag={drag:.2f}")

if __name__ == "__main__":
    main()


Generation 0: {'avg': -60.693482445991194, 'std': 65.27081519075351, 'min': -190.74688425634963, 'max': 3.897491697575343}
Generation 1: {'avg': -59.95155603772544, 'std': 63.40238659577911, 'min': -190.74688425634963, 'max': 3.1245512005849854}
Generation 2: {'avg': -58.608801403450364, 'std': 61.642793835144055, 'min': -172.08551953861925, 'max': 2.5979408347953945}
Generation 3: {'avg': -60.472874015719135, 'std': 63.21137204023404, 'min': -147.4932338449405, 'max': 2.085471676850916}
Generation 4: {'avg': -59.567156835657435, 'std': 62.20148084456366, 'min': -147.4932338449405, 'max': 2.14139290930502}
Generation 5: {'avg': -60.24213269668499, 'std': 62.93503716201551, 'min': -147.4932338449405, 'max': 2.4342110344108683}
Generation 6: {'avg': -59.553197316900835, 'std': 62.42926831982717, 'min': -147.4932338449405, 'max': 2.5896205322451418}
Generation 7: {'avg': -58.492775958056356, 'std': 61.356489719414995, 'min': -147.4932338449405, 'max': 2.477570014662002}
Generation 8: {'av